<a href="https://colab.research.google.com/github/Snowdenstyll/keno/blob/main/jupyterNotebooks/Keno_Lottery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv("2024.csv")

In [3]:
df.head()
df.tail()
df.info()
df.describe()
df.drop(['PlayDate', 'AP'], axis=1, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302 entries, 0 to 301
Data columns (total 22 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   PlayDate  302 non-null    object
 1   AP        302 non-null    object
 2   N01       302 non-null    int64 
 3   N02       302 non-null    int64 
 4   N03       302 non-null    int64 
 5   N04       302 non-null    int64 
 6   N05       302 non-null    int64 
 7   N06       302 non-null    int64 
 8   N07       302 non-null    int64 
 9   N08       302 non-null    int64 
 10  N09       302 non-null    int64 
 11  N10       302 non-null    int64 
 12  N11       302 non-null    int64 
 13  N12       302 non-null    int64 
 14  N13       302 non-null    int64 
 15  N14       302 non-null    int64 
 16  N15       302 non-null    int64 
 17  N16       302 non-null    int64 
 18  N17       302 non-null    int64 
 19  N18       302 non-null    int64 
 20  N19       302 non-null    int64 
 21  N20       302 no

In [4]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)

In [5]:
# All our games
number_of_rows = df.values.shape[0]
# Amount of games we need to take into consideration for prediction
window_length = 10
number_of_features = df.values.shape[1]

X = np.empty([ number_of_rows - window_length, window_length, number_of_features], dtype=float)
y = np.empty([ number_of_rows - window_length, number_of_features], dtype=float)


In [6]:
for i in range(0, number_of_rows-window_length):
    X[i] = transformed_df.iloc[i : i+window_length, 0 : number_of_features]
    y[i] = transformed_df.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [7]:
# Recurrent Neural Netowrk (RNN) with Long Short Term Memory (LSTM)
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout
#batch_size = 40
batch_size = 60

# Initialising the RNN
model = Sequential()
# Adding the input layer and the LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a first Dropout layer
model.add(Dropout(0.2))
# Adding a second LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a second Dropout layer
model.add(Dropout(0.2))
# Adding a third LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a fourth LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = False)))
# Adding a fourth Dropout layer
model.add(Dropout(0.2))
# Adding the first output layer
model.add(Dense(70))
# Adding the last output layer
model.add(Dense(number_of_features))

In [8]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

model.compile(optimizer=Adam(learning_rate=0.0001), loss ='mse', metrics=['accuracy'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.70, random_state=70)

# Train the model
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=400, verbose=2)
#history = model.fit(X_train, y_train, batch_size=100, epochs=600, verbose=2)

Epoch 1/400
2/2 - 18s - loss: 1.1287 - accuracy: 0.0575 - 18s/epoch - 9s/step
Epoch 2/400
2/2 - 0s - loss: 1.1210 - accuracy: 0.0575 - 49ms/epoch - 24ms/step
Epoch 3/400
2/2 - 0s - loss: 1.1116 - accuracy: 0.1149 - 45ms/epoch - 23ms/step
Epoch 4/400
2/2 - 0s - loss: 1.1067 - accuracy: 0.1494 - 49ms/epoch - 24ms/step
Epoch 5/400
2/2 - 0s - loss: 1.0995 - accuracy: 0.1034 - 39ms/epoch - 19ms/step
Epoch 6/400
2/2 - 0s - loss: 1.0917 - accuracy: 0.1379 - 36ms/epoch - 18ms/step
Epoch 7/400
2/2 - 0s - loss: 1.0842 - accuracy: 0.2069 - 39ms/epoch - 19ms/step
Epoch 8/400
2/2 - 0s - loss: 1.0757 - accuracy: 0.2184 - 38ms/epoch - 19ms/step
Epoch 9/400
2/2 - 0s - loss: 1.0700 - accuracy: 0.2069 - 36ms/epoch - 18ms/step
Epoch 10/400
2/2 - 0s - loss: 1.0621 - accuracy: 0.1839 - 34ms/epoch - 17ms/step
Epoch 11/400
2/2 - 0s - loss: 1.0517 - accuracy: 0.1839 - 31ms/epoch - 16ms/step
Epoch 12/400
2/2 - 0s - loss: 1.0443 - accuracy: 0.1494 - 30ms/epoch - 15ms/step
Epoch 13/400
2/2 - 0s - loss: 1.0376 - 

In [9]:
loss, accuracy = model.evaluate(X, y, verbose=1)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

10/10 [==============================] - 3s 14ms/step - loss: 1.3829 - accuracy: 0.2158
Test Loss: 1.3829171657562256, Test Accuracy: 0.21575342118740082
7/7 [==============================] - 0s 10ms/step - loss: 1.9553 - accuracy: 0.0439
Test Loss: 1.9552868604660034, Test Accuracy: 0.043902438133955


In [10]:
#history = model.fit(X_train, y_train, batch_size=50, epochs=100, verbose=2)
#history = model.fit(X_train, y_train, batch_size=50, epochs=100, verbose=2)

In [11]:
''' import datetime
import pytz

# Get the current time in a specific timezone
tz = pytz.timezone('America/New_York')
current_day = datetime.datetime.now(tz)

time_label = "Midday" if (current_day.hour <= 14) else "Evening"

label = current_day.strftime("%Y_%m_%d_") + time_label
model.save(f'Models/{label}.keras')
print("saved model") '''

' import datetime\nimport pytz\n\n# Get the current time in a specific timezone\ntz = pytz.timezone(\'America/New_York\')\ncurrent_day = datetime.datetime.now(tz)\n\ntime_label = "Midday" if (current_day.hour <= 14) else "Evening"\n\nlabel = current_day.strftime("%Y_%m_%d_") + time_label\nmodel.save(f\'Models/{label}.keras\')\nprint("saved model") '

In [12]:
df1 = df.copy()
to_predict = df1.tail(window_length)
to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)
to_predict

<ipython-input-12-efb52fef0daf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)


,N01,N02,N03,N04,N05,N06,N07,N08,N09,N10,N11,N12,N13,N14,N15,N16,N17,N18,N19,N20
292,1,5,6,10,11,13,19,23,37,42,43,46,50,52,55,57,60,61,63,65
293,3,5,9,29,30,31,32,34,35,40,47,50,52,54,56,64,65,67,69,70
294,1,2,4,7,11,13,17,28,30,36,41,42,44,52,54,59,60,64,65,69
295,5,7,8,13,14,18,24,27,33,37,40,56,57,58,62,64,65,68,69,70
296,1,2,3,5,9,16,20,24,30,32,37,42,43,46,47,48,56,62,65,67
297,1,6,9,12,13,16,19,23,26,34,40,44,46,50,51,56,61,65,66,70
298,2,5,6,9,18,19,21,25,26,30,31,33,34,44,48,50,60,65,67,69
299,8,11,17,18,24,26,30,31,33,38,39,44,45,46,48,51,54,63,64,65
300,1,3,7,9,10,12,14,15,19,22,25,26,31,35,36,45,51,54,56,65


In [13]:
prediction = np.array(df.tail(1))

x_next = scaler.transform(to_predict)
y_pred = model.predict(np.array([x_next]))

temp = np.array(np.unique(scaler.inverse_transform(y_pred).astype(int)[0]))
combined_array = np.concatenate((temp, np.array(prediction[0])))
winning_numbers = np.bincount(combined_array)
winners = np.unique(combined_array)

uniques, uniq_idx, counts = np.unique(combined_array,return_index=True,return_counts=True)
duplicates = combined_array[uniq_idx[counts>=2] ]  # <--- Get duplicates

top_5_indices = np.argsort(y_pred[0], axis=0)[::-1][:5]

top_5_numbers = []

winning_indices = np.where(np.isin(prediction[0], duplicates))[0]

# Print the winning indices

for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    top_5_numbers.append(number)

print("The top 5 indices:", top_5_indices)
print("The top 5 numbers:", top_5_numbers)
print("\n")
top_10_indices = np.argsort(y_pred[0], axis=0)[::-1][:10]
top_10_numbers = []

winning_indices = np.where(np.isin(prediction[0], duplicates))[0]

for index in top_10_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    top_10_numbers.append(number)

print("The top 10 indices:", top_10_indices)
print("The top 10 numbers:", top_10_numbers)
print("\n")

print("The predicted numbers for the lottery game are:  ", temp)
print("The actual numbers in the last lottery game were:", prediction[0])
print("\n")
print("The winning indices:", winning_indices)
print("winners:", duplicates)
print(len(duplicates)/20)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


1/1 [==============================] - 5s 5s/step
The top 5 indices: [3 4 8 5 7]
The top 5 numbers: [20, 21, 34, 22, 29]


The top 10 indices: [ 3  4  8  5  7  0  6 10  9 11]
The top 10 numbers: [20, 21, 34, 22, 29, 4, 24, 37, 36, 39]


The predicted numbers for the lottery game are:   [ 4  6  9 21 22 24 26 30 35 39 40 42 44 45 52 57]
The actual numbers in the last lottery game were: [ 4  5  8 20 21 22 24 29 34 36 37 39 40 41 43 45 46 52 53 55]


The winning indices: [ 0  4  5  6 11 12 15 17]
winners: [ 4 21 22 24 39 40 45 52]
0.4


# Predict the next set of numbers

In [14]:
next = df.copy()
next = next.tail(window_length)
next = np.array(next)

x_next = scaler.transform(next)
y_next_pred = model.predict(np.array([x_next]))
prediction = scaler.inverse_transform(y_next_pred).astype(int)
print("The predicted numbers for the lottery game:", temp)
print("\n")
top_5_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:5]
top_5_numbers = []
for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    top_5_numbers.append(number)

print("The top 5 indices", top_5_indices)
print("The top 5 numbers:", top_5_numbers)
print("\n")

winning_numbers = []
for index in winning_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    winning_numbers.append(number)

top_10_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:10]
top_10_numbers = []
for index in top_10_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_10_numbers.append(number)
print("The top 10 indices", top_10_indices)
print("The top 10 numbers:", top_10_numbers)
print("\n")

print("winning indices", winning_indices)
print("winning numbers", winning_numbers)


1/1 [==============================] - 3s 3s/step
The predicted numbers for the lottery game: [ 4  6  9 21 22 24 26 30 35 39 40 42 44 45 52 57]


The top 5 indices [3 0 4 2 5]
The top 5 numbers: [21, 5, 22, 13, 23]


The top 10 indices [ 3  0  4  2  5  1  6  8  7 10]
The top 10 numbers: [21, 5, 22, 13, 23, 8, 25, 31, 26, 35]


winning indices [ 0  4  5  6 11 12 15 17]
winning numbers [5, 22, 23, 25, 36, 40, 47, 50]


In [15]:
next = df.copy()
next = next.tail(5)
next = np.array(next)

x_next = scaler.transform(next)
y_next_pred = model.predict(np.array([x_next]))

# Get the actual values of the top 10 predictions
prediction = scaler.inverse_transform(y_next_pred).astype(int)

# Get the indices of the top 10 predictions (change axis=0 if predictions are along a different axis)
top_10_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:10]
top_5_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:5]

top_10_numbers = []
top_5_numbers = []

# index [0,2,4] usually are correct
# Loop through each index in top_10_indices
for index in top_10_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_10_numbers.append(number)

for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_5_numbers.append(number)

winning_numbers = []
for index in winning_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    winning_numbers.append(number)


print("The top 5 indices", top_5_indices)
print("The top 5 numbers:", top_5_numbers)
print("\n")

print("The top 10 indices", top_10_indices)
print("The top 10 numbers:", top_10_numbers)
print("\n")

print("winning indices", winning_indices)
print("winning numbers", winning_numbers)
print("\n")

print("The predicted numbers:", prediction)


1/1 [==============================] - 0s 22ms/step
The top 5 indices [ 0 17 13  2  1]
The top 5 numbers: [3, 60, 47, 10, 7]


The top 10 indices [ 0 17 13  2  1  3  8 16 12 14]
The top 10 numbers: [3, 60, 47, 10, 7, 14, 30, 57, 44, 50]


winning indices [ 0  4  5  6 11 12 15 17]
winning numbers [3, 17, 20, 23, 40, 44, 54, 60]


The predicted numbers: [[ 3  7 10 14 17 20 23 27 30 33 37 40 44 47 50 54 57 60 63 66]]
